In [1]:
import modi_plus
import cv2
import time
import os
import glob

# photos 폴더가 없으면 생성, 있으면 내부 파일 모두 삭제
if not os.path.exists("photos"):
    os.makedirs("photos")
else:
    files = glob.glob("photos/*")
    for f in files:
        os.remove(f)

Running PyMODI+ (v0.3.1)


In [2]:
bundle = modi_plus.MODIPlus()
button = bundle.buttons[0]
display = bundle.displays[0]
led = bundle.leds[0]

guide_texts = [
    "튼튼한 식탁/책상 아래를 찍어주세요!",
    "구급상자 내용물을 펼쳐서 찍어주세요!",
    "가장 큰 가구와 벽이 만나는 부분을 찍어주세요!",
    "현관문으로 가는 길 전체를 찍어주세요!",
    "비상 식량과 물을 모두 찍어주세요!",
    "침대 옆 바닥을 찍어주세요!",
    "가장 높은 선반 위를 찍어주세요!",
    "가스 밸브 주변을 찍어주세요!"
]

Initiating serial connection...
Serial is open at "COM4"
Start initializing connected MODI+ modules
Network(0x93C) has been connected!
Display(0x238) has been connected!
MODI+ modules are initialized!
Led(0x171) has been connected!
Button(0x36A) has been connected!


In [3]:
display.text = "웹캠 연결중..."
cap = cv2.VideoCapture(1)

time.sleep(1)
ret, frame = cap.read()
if ret:
    print("웹캠 연결됨")
    display.text = "웹캠 연결 완료"
    time.sleep(3)
    display.text = ""
else:
    print("웹캠 연결 안됨")
    display.text = "웹캠 연결 실패"
    cap.release()
    exit()


웹캠 연결됨


In [4]:
for idx, guide in enumerate(guide_texts):
    display.text = guide
    led.rgb = (0, 10, 0)  # 초록색
    print(f"안내: {guide}")
    while True:
        if button.clicked:
            ret, frame = cap.read()
            if ret:
                filename = f"photos/photo_{idx+1}_{int(time.time())}.png"
                cv2.imwrite(filename, frame)
                print(f"{filename} 저장됨")
            time.sleep(0.5)  # 버튼 중복 방지
            break
        time.sleep(0.1)
    led.rgb = (0, 0, 0)  # LED 끄기

display.text = "촬영 완료!"
print("모든 사진 촬영 완료")
time.sleep(3)
display.text = ""
cap.release()


안내: 튼튼한 식탁/책상 아래를 찍어주세요!
photos/photo_1_1754124309.png 저장됨
안내: 구급상자 내용물을 펼쳐서 찍어주세요!
photos/photo_2_1754124312.png 저장됨
안내: 가장 큰 가구와 벽이 만나는 부분을 찍어주세요!
photos/photo_3_1754124317.png 저장됨
안내: 현관문으로 가는 길 전체를 찍어주세요!
photos/photo_4_1754124320.png 저장됨
안내: 비상 식량과 물을 모두 찍어주세요!
photos/photo_5_1754124321.png 저장됨
안내: 침대 옆 바닥을 찍어주세요!
photos/photo_6_1754124324.png 저장됨
안내: 가장 높은 선반 위를 찍어주세요!
photos/photo_7_1754124327.png 저장됨
안내: 가스 밸브 주변을 찍어주세요!
photos/photo_8_1754124331.png 저장됨
모든 사진 촬영 완료


In [5]:
!pip install firebase-admin

import base64
import firebase_admin
from firebase_admin import credentials, firestore
import glob

# Firebase Admin SDK 초기화 (이미 되어있다면 생략)
cred = credentials.Certificate("./safeguard-ai-qoucc-firebase-adminsdk-fbsvc-82e83e0c1a.json")
firebase_admin.initialize_app(cred)

db = firestore.client()

# photos 폴더에서 8장 사진 경로 가져오기 (파일명 순서대로)
photo_paths = sorted(glob.glob("photos/photo_*.png"))[:8]
photo_data_uris = []

for path in photo_paths:
    with open(path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        data_uri = f"data:image/png;base64,{encoded_string}"
        photo_data_uris.append(data_uri)

# Firestore 문서 업데이트
doc_ref = db.collection('diagnosis').document('latest')
doc_ref.update({
    'photoDataUris': photo_data_uris,
    'status': 'captured'
})

print("8장의 사진이 Firestore에 업로드되었습니다.")


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


PermissionDenied: 403 Cloud Firestore API has not been used in project safeguard-ai-qoucc before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/firestore.googleapis.com/overview?project=safeguard-ai-qoucc then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "firestore.googleapis.com"
}
metadata {
  key: "serviceTitle"
  value: "Cloud Firestore API"
}
metadata {
  key: "containerInfo"
  value: "safeguard-ai-qoucc"
}
metadata {
  key: "consumer"
  value: "projects/safeguard-ai-qoucc"
}
metadata {
  key: "activationUrl"
  value: "https://console.developers.google.com/apis/api/firestore.googleapis.com/overview?project=safeguard-ai-qoucc"
}
, locale: "en-US"
message: "Cloud Firestore API has not been used in project safeguard-ai-qoucc before or it is disabled. Enable it by visiting https://console